Copyright (C) Microsoft Corporation.

# PreProcessing of Images for TF OD API  

## Introduction

This notebook will get the data ready for training.

In this notebook you will
* Tranform the test data set into TensorFlow records.
* Add the TensorFlow records to the datastore.


In this notebook, we create tensorflow records for training and validation datasets, details can be found [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md)

In [ ]:
import sys
import os
import io
import glob
from PIL import Image
import random
from lxml import etree
import contextlib2
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from IPython.display import display
from utilities import create_dir
random.seed(4)

import tensorflow as tf
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from object_detection.dataset_tools import tf_record_creation_util

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
# Annotated Images directory
data_dir = './dataset'

# Label map path
label_map_path = './outofstock_label_map.pbtxt'

# Outputs path
output_dir = 'tfrecords'

In [ ]:
create_dir(output_dir)

In [ ]:
# Read label dictionary
label_map_dict = label_map_util.get_label_map_dict(label_map_path)
class_name = 'outofstock'

In [ ]:
# Tensorflow record output paths
train_output_path = os.path.join(output_dir, 'outofstock_train.record')
val_output_path = os.path.join(output_dir, 'outofstock_val.record')

Gets the total number of images for processing.

In [ ]:
in_images = glob.glob(data_dir + '/**/'+ '*.jpg')
print("There are {} images.".format(len(in_images)))

Gets the number of images in the validation dataset.

In [ ]:
val_images =  glob.glob(data_dir + '/test/'+ '*.jpg')
print("There are {} images.".format(len(val_images)))

Gets the number of images in the training dataset.

In [ ]:
train_images =  glob.glob(data_dir + '/train/'+ '*.jpg')
print("There are {} images.".format(len(train_images)))

In [ ]:
# Sample of training images
for img_path in train_images[:5]:
    img = Image.open(img_path)
    display(img)
    print(img.size)

In [ ]:
# Sample of validation images
for img_path in val_images[:5]:
    img = Image.open(img_path)
    display(img)
    print(img.size)

Changes the XML annotation and image into a format TensorFlow can handle.

In [ ]:
def dict_to_tf_example(data, label_map_dict, img_path):
    with tf.Graph().as_default():
        image_contents = tf.read_file(img_path)
        image = tf.image.decode_jpeg(image_contents, channels=3)
        init_op = tf.initialize_all_tables()
        with tf.Session() as sess:
            sess.run(init_op)
            try:
                tmp = sess.run(image)
            except InvalidArgumentError:
                print(img_path)
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    width = int(data['size']['width'])
    height = int(data['size']['height'])
    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    classes = []
    classes_text = []

    if 'object' in data:
        for obj in data['object']:
            xmin = float(obj['bndbox']['xmin'])
            xmax = float(obj['bndbox']['xmax'])
            ymin = float(obj['bndbox']['ymin'])
            ymax = float(obj['bndbox']['ymax'])

            xmins.append(xmin / width)
            ymins.append(ymin / height)
            xmaxs.append(xmax / width)
            ymaxs.append(ymax / height)
            classes_text.append(class_name.encode('utf8'))
            classes.append(label_map_dict[class_name])   
    feature_dict = {
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(data['filename'].encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(data['filename'].encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes)
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example

Creates the actual TensorFlow records for the training and validation datasets.

In [ ]:
def create_tf_record(output_filename, num_shards, label_map_dict, examples):
    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_filename, num_shards)
        for idx, example in enumerate(examples):
            if idx % 50 == 0:
                print('On image {} of {}'.format(idx, len(examples)))
            xml_path = os.path.join(example[:-4] + '.xml')
            if not os.path.exists(xml_path):
                print('Could not find {}, ignoring example'.format(xml_path))
                continue
            with tf.gfile.GFile(xml_path, 'r') as fid:
                xml_str = fid.read()
            xml = etree.fromstring(xml_str)
            data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
            try:
                tf_example = dict_to_tf_example(data, label_map_dict, example)
                if tf_example:
                    shard_idx = idx % num_shards
                    output_tfrecords[shard_idx].write(tf_example.SerializeToString())
            except ValueError:
                print('Invalid example: {}, ignoring.'.format(xml_path))

In [ ]:
# Create tensorflow records for training
create_tf_record(train_output_path, 2, label_map_dict, train_images)

In [ ]:
 # Create tensorflow records for validation
create_tf_record(val_output_path, 1, label_map_dict, val_images)

Adds the TensorFlow records to the datastore.

In [ ]:
ws = Workspace.from_config()
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=output_dir, target_path='./tfdataset',
          overwrite=True, show_progress=True)

Next, you can move to training the model.